# OpenParticle

The main classes of this 'package' are `FockState` and `ParticleOperator`. Their functionality is as follows.

In [1]:
#Imports
from FockState import *
from ParticleOperator import *

## `FockState` Class

A Fock state in second quantization is a quantum state given by an occupancy representation of some particular mode(s). In quantum field theory, we work mainly in momentum space, so these modes represent the occupied momentum of some particle. For example, for a particle with potential momentum modes $p \in \{0, 1, 2, 3, \dots, N\}$, the corresponding Fock state vector for a particle in momentum mode $2$ would be $|0, 0, 1, 0, \dots, 0 \rangle$

In general, we are interested in hadrons, or bound states of Quantum Chromodynamics (QCD). Because of this, we need a representation of our hadronic states in terms of fermions (quarks), antifermions (antiquarks), and bosons (gluons). The way this Fock state is written is: $|f;\bar{f}; b\rangle = |f\rangle \otimes |\bar{f}\rangle\otimes |b\rangle$, where each particle type has its own representation as shown above. Fermions and antifermions obey fermi statistics such that the occupancy of fermions in a particular mode, $n^{f, \bar{f}}_i \in \{0, 1\}$, while gluons obey bose statistics with occupancies in a given mode, $n^b_i \in \{0, 1, 2, 3, \cdots \}$

To define your Fock state, you need to pass in a vector corresponding to occupied fermion modes, occupied antifermion modes, and occupied bosonic modes designated how many bosons exist in each mode. The four parameters to pass when instantiating the class are `ferm_occupancy`, `antiferm_occupancy`, `bos_occupancy`, and `coeff`. The first three are lists of length $N$ where each non-zero entry denotes a particle of the particular type to have the corresponding momentum orbital occupied, with the non-zero number representing the occupancy number. `coeff` is a parameter specifying the coefficient in front of the Fock state. 

In [2]:
#e.g. N = 3

f_occ = [1, 0, 0]
af_occ = [0, 0, 1]
b_occ = [4, 0, 0]

x = FockState(ferm_occupancy = f_occ, antiferm_occupancy = af_occ,
              bos_occupancy = b_occ, coeff = 1.0)

print(x)

1.0 * |1,0,0; 0,0,1; 4,0,0⟩


### `FockState` Methods:

- `compact_state()`:

We can also print a more compact encoding, which will be useful when N is very large. The `.compact_state()` method will accomplish this. For fermions and antifermions, the values in the compact encoding correspond to the occupied modes while for bosons, it returns a tuple $(i, n^b_i)$ in order to display information about how many bosons are in each mode. 

In [3]:
x.compact_state()

'|0; 2; (0, 4)⟩'

- `dagger()`: 

Returns the bra corresponding to the Fock state ket.

In [4]:
print(x.dagger())

1.0 * ⟨1,0,0; 0,0,1; 4,0,0|


### Related Classes to FockState

### `FockStateSum`

You can add Fock states the same way that you add kets. Note that you need to make sure the number of modes is the same for each state. 

In [5]:
state_1 = FockState([1], [0], [0], 1)
state_2 = FockState([0], [1], [1], 1)
state = state_1 + state_2
print(state)

1 * |1; 0; 0⟩ + 1 * |0; 1; 1⟩


### `ConjugateFockState`

`ConjugateFockState` is instantiated in two ways. The first is identical to how `FockState` is instantiated. The second is via the method `.from_state(state: FockState)`, which essentially turns a ket into a bra. 

In [6]:
y = ConjugateFockState([1, 0, 0], [0, 0, 1], [4, 0, 0], 1)
z = ConjugateFockState.from_state(x)
print(y)
print(z)

1 * ⟨1,0,0; 0,0,1; 4,0,0|
1.0 * ⟨1,0,0; 0,0,1; 4,0,0|


With the `ConjugateFockState` class defined, we can now take inner products via `*` or with `conj_state.inner_product(state)`

In [9]:
print(y * z)
print(y.inner_product(z))

1.0
1.0


## `ParticleOperator` Class